In [38]:
!pip install transformers==4.3.2
import pandas as pd
import torch
import io
import torch.nn.functional as F
import random
import numpy as np
import time
import math
import datetime
import torch.nn as nn
from transformers import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install sentencepiece

##Set random values
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(seed_val)

You should consider upgrading via the 'c:\users\komol\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [39]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1650


In [40]:
#--------------------------------
#  Transformer parameters
#--------------------------------
max_seq_length = 64
batch_size = 4

#--------------------------------
#  GAN-BERT specific parameters
#--------------------------------
# number of hidden layers in the generator, 
# each of the size of the output space
num_hidden_layers_g = 1; 
# number of hidden layers in the discriminator, 
# each of the size of the input space
num_hidden_layers_d = 1; 
# size of the generator's input noisy vectors
noise_size = 100
# dropout to be applied to discriminator's input vectors
out_dropout_rate = 0.2

# Replicate labeled data to balance poorly represented datasets, 
# e.g., less than 1% of labeled material
apply_balance = True

#--------------------------------
#  Optimization parameters
#--------------------------------
learning_rate_discriminator = 5e-5
learning_rate_generator = 5e-5
epsilon = 1e-8
num_train_epochs = 1
multi_gpu = True
# Scheduler
apply_scheduler = False
warmup_proportion = 0.1
# Print
print_each_n_step = 10

#--------------------------------
#  Adopted Tranformer model
#--------------------------------
# Since this version is compatible with Huggingface transformers, you can uncomment
# (or add) transformer models compatible with GAN

model_name = "bert-base-cased"
#model_name = "bert-base-uncased"
#model_name = "roberta-base"
#model_name = "albert-base-v2"
#model_name = "xlm-roberta-base"
#model_name = "amazon/bort"

#--------------------------------
#  Retrieve the TREC QC Dataset
#--------------------------------
! git clone https://github.com/crux82/ganbert

#  NOTE: in this setting 50 classes are involved
#labeled_file = "./ganbert/data/labeled.tsv"
#unlabeled_file = "./ganbert/data/unlabeled.tsv"
#test_filename = "./ganbert/data/test.tsv"

# label_list = ["UNK_UNK","ABBR_abb", "ABBR_exp", "DESC_def", "DESC_desc", 
#               "DESC_manner", "DESC_reason", "ENTY_animal", "ENTY_body", 
#               "ENTY_color", "ENTY_cremat", "ENTY_currency", "ENTY_dismed", 
#               "ENTY_event", "ENTY_food", "ENTY_instru", "ENTY_lang", 
#               "ENTY_letter", "ENTY_other", "ENTY_plant", "ENTY_product", 
#               "ENTY_religion", "ENTY_sport", "ENTY_substance", "ENTY_symbol", 
#               "ENTY_techmeth", "ENTY_termeq", "ENTY_veh", "ENTY_word", "HUM_desc", 
#               "HUM_gr", "HUM_ind", "HUM_title", "LOC_city", "LOC_country", 
#               "LOC_mount", "LOC_other", "LOC_state", "NUM_code", "NUM_count", 
#               "NUM_date", "NUM_dist", "NUM_money", "NUM_ord", "NUM_other", 
#               "NUM_perc", "NUM_period", "NUM_speed", "NUM_temp", "NUM_volsize", 
#               "NUM_weight"]


fatal: destination path 'ganbert' already exists and is not an empty directory.


In [41]:
import os
# os.getcwd()

#!cp gdrive/MyDrive/Colab\ Notebooks/Corona_NLP_test.csv ganbert/data/
#!cp gdrive/MyDrive/Colab\ Notebooks/Corona_NLP_train.csv ganbert/data/

In [42]:
df_train = pd.read_csv(r'C:\Users\Komol\one drive\OneDrive - Queensland University of Technology\H-Migrated\Jupyter_Code_ipynb\Research Officer SEF\train.csv', encoding='latin-1')

df_test = pd.read_csv(r'C:\Users\Komol\one drive\OneDrive - Queensland University of Technology\H-Migrated\Jupyter_Code_ipynb\Research Officer SEF\test.csv', encoding='latin-1')
df_unlabeled= pd.read_csv(r'C:\Users\Komol\one drive\OneDrive - Queensland University of Technology\H-Migrated\Jupyter_Code_ipynb\Research Officer SEF\Unlabelled\50_200_selected_combined.csv', encoding='latin-1')
df_train.head()
print(df_train.label.unique())
#df_train = df_train.sample(frac=0.1)
df_train.shape

[1 0]


(4000, 2)

In [43]:
#label_list = ['Neutral', 'Positive', 'Extremely Negative', 'Negative', 'Extremely Positive','UNK']
label_list = [0,1,2]

In [44]:
df_unlabeled=df_unlabeled.drop(['id'],axis='columns')

In [45]:
# Change each of the sentiment for unlabeled data to Unknown
for i in df_unlabeled.index:
    df_unlabeled.at[i, "label"] = '2'

In [46]:
df_unlabeled

,label,text
0,2,i love how there always seems to be an outage ...
1,2,xrtu: it's time to decide once and for all whi...
2,2,xrtu: it's never too late to dream.
3,2,maga has ruined all of us â ï¸
4,2,xrtu: lord guide and protect the entire caribb...
...,...,...
38266,2,xrtu: please stop what you're doing right now ...
38267,2,"baby you so fine to me, but i ain't tryna love..."
38268,2,bitch i'm still alive?? xurl
38269,2,xatp no problem. will do.


In [47]:
#df_train_for_ganbert = df_train.sample(frac=0.01) # use 1% of the labeled data for training
#df_unlabeled = df_train.drop(df_train_for_ganbert.index)

print(df_train.shape,df_test.shape, df_unlabeled.shape)

(4000, 2) (1000, 2) (38271, 2)


In [51]:
# Defining function to format the dataset to be used in the dataloader
def get_examples(df):

    examples = []

    # iterate through each row
    for index, row in df.iterrows():
        examples.append((row['text'], row['label']))

    return examples

In [52]:
labeled_examples = get_examples(df_train)
unlabeled_examples = get_examples(df_unlabeled)
test_examples = get_examples(df_test)

In [53]:
transformer = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [23]:
torch.save(transformer, 'transformer')

In [24]:
transformer.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [54]:
def generate_data_loader(input_examples, label_masks, label_map, do_shuffle = False, balance_label_examples = False):
  '''
  Generate a Dataloader given the input examples, eventually masked if they are 
  to be considered NOT labeled.
  '''
  examples = []

  # Count the percentage of labeled examples  
  num_labeled_examples = 0
  for label_mask in label_masks:
    if label_mask: 
      num_labeled_examples += 1
  label_mask_rate = num_labeled_examples/len(input_examples)

  # if required it applies the balance
  for index, ex in enumerate(input_examples): 
    if label_mask_rate == 1 or not balance_label_examples:
      examples.append((ex, label_masks[index]))
    else:
      # IT SIMULATE A LABELED EXAMPLE
      if label_masks[index]:
        balance = int(1/label_mask_rate)
        balance = int(math.log(balance,2))
        if balance < 1:
          balance = 1
        for b in range(0, int(balance)):
          examples.append((ex, label_masks[index]))
      else:
        examples.append((ex, label_masks[index]))
  
  #-----------------------------------------------
  # Generate input examples to the Transformer
  #-----------------------------------------------
  input_ids = []
  input_mask_array = []
  label_mask_array = []
  label_id_array = []

  # Tokenization 
  for (text, label_mask) in examples:
    encoded_sent = tokenizer.encode(text[0], add_special_tokens=True, max_length=max_seq_length, padding="max_length", truncation=True)
    input_ids.append(encoded_sent)
    label_id_array.append(label_map[text[1]])
    label_mask_array.append(label_mask)
  
  # Attention to token (to ignore padded input wordpieces)
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]                          
    input_mask_array.append(att_mask)
  # Convertion to Tensor
  input_ids = torch.tensor(input_ids) 
  input_mask_array = torch.tensor(input_mask_array)
  label_id_array = torch.tensor(label_id_array, dtype=torch.long)
  label_mask_array = torch.tensor(label_mask_array)

  # Building the TensorDataset
  dataset = TensorDataset(input_ids, input_mask_array, label_id_array, label_mask_array)

  if do_shuffle:
    sampler = RandomSampler
  else:
    sampler = SequentialSampler

  # Building the DataLoader
  return DataLoader(
              dataset,  # The training samples.
              sampler = sampler(dataset), 
              batch_size = batch_size, # Trains with this batch size.
              drop_last=True) 

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [55]:
label_map = {}
for (i, label) in enumerate(label_list):
  label_map[label] = i
#------------------------------
#   Load the train dataset
#------------------------------
train_examples = labeled_examples
#The labeled (train) dataset is assigned with a mask set to True
train_label_masks = np.ones(len(labeled_examples), dtype=bool)
#If unlabel examples are available
if unlabeled_examples:
  train_examples = train_examples + unlabeled_examples
  #The unlabeled (train) dataset is assigned with a mask set to False
  tmp_masks = np.zeros(len(unlabeled_examples), dtype=bool)
  train_label_masks = np.concatenate([train_label_masks,tmp_masks])

train_dataloader = generate_data_loader(train_examples, train_label_masks, label_map, do_shuffle = True, balance_label_examples = apply_balance)

#------------------------------
#   Load the test dataset
#------------------------------
#The labeled (test) dataset is assigned with a mask set to True
test_label_masks = np.ones(len(test_examples), dtype=bool)

test_dataloader = generate_data_loader(test_examples, test_label_masks, label_map, do_shuffle = False, balance_label_examples = False)

In [56]:
#------------------------------
#   The Generator as in 
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Generator(nn.Module):
    def __init__(self, noise_size=100, output_size=512, hidden_sizes=[512], dropout_rate=0.1):
        super(Generator, self).__init__()
        layers = []
        hidden_sizes = [noise_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        layers.append(nn.Linear(hidden_sizes[-1],output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, noise):
        output_rep = self.layers(noise)
        return output_rep

#------------------------------
#   The Discriminator
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Discriminator(nn.Module):
    def __init__(self, input_size=512, hidden_sizes=[512], num_labels=2, dropout_rate=0.1):
        super(Discriminator, self).__init__()
        self.input_dropout = nn.Dropout(p=dropout_rate)
        layers = []
        hidden_sizes = [input_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        self.layers = nn.Sequential(*layers) #per il flatten
        self.logit = nn.Linear(hidden_sizes[-1],num_labels+1) # +1 for the probability of this sample being fake/real.
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, input_rep):
        input_rep = self.input_dropout(input_rep)
        last_rep = self.layers(input_rep)
        logits = self.logit(last_rep)
        probs = self.softmax(logits)
        return last_rep, logits, probs

In [57]:
# The config file is required to get the dimension of the vector produced by 
# the underlying transformer
config = AutoConfig.from_pretrained(model_name)
hidden_size = int(config.hidden_size)
# Define the number and width of hidden layers
hidden_levels_g = [hidden_size for i in range(0, num_hidden_layers_g)]
hidden_levels_d = [hidden_size for i in range(0, num_hidden_layers_d)]

#-------------------------------------------------
#   Instantiate the Generator and Discriminator
#-------------------------------------------------
generator = Generator(noise_size=noise_size, output_size=hidden_size, hidden_sizes=hidden_levels_g, dropout_rate=out_dropout_rate)
discriminator = Discriminator(input_size=hidden_size, hidden_sizes=hidden_levels_d, num_labels=len(label_list), dropout_rate=out_dropout_rate)

# Put everything in the GPU if available
if torch.cuda.is_available():    
  generator.cuda()
  discriminator.cuda()
  transformer.cuda()
  if multi_gpu:
    transformer = torch.nn.DataParallel(transformer)
print(config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.3.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



In [58]:
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

#models parameters
transformer_vars = [i for i in transformer.parameters()]
d_vars = transformer_vars + [v for v in discriminator.parameters()]
g_vars = [v for v in generator.parameters()]

#optimizer
dis_optimizer = torch.optim.AdamW(d_vars, lr=learning_rate_discriminator)
gen_optimizer = torch.optim.AdamW(g_vars, lr=learning_rate_generator) 

#scheduler
if apply_scheduler:
  num_train_examples = len(train_examples)
  num_train_steps = int(num_train_examples / batch_size * num_train_epochs)
  num_warmup_steps = int(num_train_steps * warmup_proportion)

  scheduler_d = get_constant_schedule_with_warmup(dis_optimizer, 
                                           num_warmup_steps = num_warmup_steps)
  scheduler_g = get_constant_schedule_with_warmup(gen_optimizer, 
                                           num_warmup_steps = num_warmup_steps)

# For each epoch...
for epoch_i in range(0, num_train_epochs):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, num_train_epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    tr_g_loss = 0
    tr_d_loss = 0

    # Put the model into training mode.
    transformer.train() 
    generator.train()
    discriminator.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every print_each_n_step batches.
        if step % print_each_n_step == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_label_mask = batch[3].to(device)
        
        # Encode real data in the Transformer
        model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
        hidden_states = model_outputs[-1]
        
        # Generate fake data that should have the same distribution of the ones
        # encoded by the transformer. 
        # First noisy input are used in input to the Generator
        noise = torch.zeros(b_input_ids.shape[0],noise_size, device=device).uniform_(0, 1)
        # Gnerate Fake data
        gen_rep = generator(noise)

        # Generate the output of the Discriminator for real and fake data.
        # First, we put together the output of the tranformer and the generator
        disciminator_input = torch.cat([hidden_states, gen_rep], dim=0)
        # Then, we select the output of the disciminator
        features, logits, probs = discriminator(disciminator_input)

        # Finally, we separate the discriminator's output for the real and fake
        # data
        features_list = torch.split(features, batch_size)
        D_real_features = features_list[0]
        D_fake_features = features_list[1]
        
        logits_list = torch.split(logits, batch_size)
        D_real_logits = logits_list[0]
        D_fake_logits = logits_list[1]
        
        probs_list = torch.split(probs, batch_size)
        D_real_probs = probs_list[0]
        D_fake_probs = probs_list[1]

        #---------------------------------
        #  LOSS evaluation
        #---------------------------------
        # Generator's LOSS estimation
        g_loss_d = -1 * torch.mean(torch.log(1 - D_fake_probs[:,-1] + epsilon))
        g_feat_reg = torch.mean(torch.pow(torch.mean(D_real_features, dim=0) - torch.mean(D_fake_features, dim=0), 2))
        g_loss = g_loss_d + g_feat_reg
  
        # Disciminator's LOSS estimation
        logits = D_real_logits[:,0:-1]
        log_probs = F.log_softmax(logits, dim=-1)
        # The discriminator provides an output for labeled and unlabeled real data
        # so the loss evaluated for unlabeled data is ignored (masked)
        label2one_hot = torch.nn.functional.one_hot(b_labels, len(label_list))
        per_example_loss = -torch.sum(label2one_hot * log_probs, dim=-1)
        per_example_loss = torch.masked_select(per_example_loss, b_label_mask.to(device))
        labeled_example_count = per_example_loss.type(torch.float32).numel()

        # It may be the case that a batch does not contain labeled examples, 
        # so the "supervised loss" in this case is not evaluated
        if labeled_example_count == 0:
          D_L_Supervised = 0
        else:
          D_L_Supervised = torch.div(torch.sum(per_example_loss.to(device)), labeled_example_count)
                 
        D_L_unsupervised1U = -1 * torch.mean(torch.log(1 - D_real_probs[:, -1] + epsilon))
        D_L_unsupervised2U = -1 * torch.mean(torch.log(D_fake_probs[:, -1] + epsilon))
        d_loss = D_L_Supervised + D_L_unsupervised1U + D_L_unsupervised2U

        #---------------------------------
        #  OPTIMIZATION
        #---------------------------------
        # Avoid gradient accumulation
        gen_optimizer.zero_grad()
        dis_optimizer.zero_grad()

        # Calculate weigth updates
        # retain_graph=True is required since the underlying graph will be deleted after backward
        g_loss.backward(retain_graph=True)
        d_loss.backward() 
        
        # Apply modifications
        gen_optimizer.step()
        dis_optimizer.step()

        # A detail log of the individual losses
        #print("{0:.4f}\t{1:.4f}\t{2:.4f}\t{3:.4f}\t{4:.4f}".
        #      format(D_L_Supervised, D_L_unsupervised1U, D_L_unsupervised2U,
        #             g_loss_d, g_feat_reg))

        # Save the losses to print them later
        tr_g_loss += g_loss.item()
        tr_d_loss += d_loss.item()

        # Update the learning rate with the scheduler
        if apply_scheduler:
          scheduler_d.step()
          scheduler_g.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss_g = tr_g_loss / len(train_dataloader)
    avg_train_loss_d = tr_d_loss / len(train_dataloader)             
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss generetor: {0:.3f}".format(avg_train_loss_g))
    print("  Average training loss discriminator: {0:.3f}".format(avg_train_loss_d))
    print("  Training epcoh took: {:}".format(training_time))

    print("Saving the models...............................")
    # Saving the model
    torch.save(transformer, 'transformer')
    torch.save(discriminator, 'discriminator')

        
    # ========================================
    #     TEST ON THE EVALUATION DATASET
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our test set.
    print("")
    print("Running Test...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    transformer.eval() #maybe redundant
    discriminator.eval()
    generator.eval()

    # Tracking variables 
    total_test_accuracy = 0
   
    total_test_loss = 0
    nb_test_steps = 0

    all_preds = []
    all_labels_ids = []

    #loss
    nll_loss = torch.nn.CrossEntropyLoss(ignore_index=-1)

    # Evaluate data for one epoch
    for batch in test_dataloader:
        
        # Unpack this training batch from our dataloader. 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        
            model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
            hidden_states = model_outputs[-1]
            _, logits, probs = discriminator(hidden_states)
            ###log_probs = F.log_softmax(probs[:,1:], dim=-1)
            filtered_logits = logits[:,0:-1]
            # Accumulate the test loss.
            total_test_loss += nll_loss(filtered_logits, b_labels)
            
        # Accumulate the predictions and the input labels
        _, preds = torch.max(filtered_logits, 1)
        all_preds += preds.detach().cpu()
        all_labels_ids += b_labels.detach().cpu()


    # Report the final accuracy for this validation run.
    all_preds = torch.stack(all_preds).numpy()
    all_labels_ids = torch.stack(all_labels_ids).numpy()
    test_accuracy = np.sum(all_preds == all_labels_ids) / len(all_preds)
    print("  Accuracy: {0:.3f}".format(test_accuracy))

    # Calculate the average loss over all of the batches.
    avg_test_loss = total_test_loss / len(test_dataloader)
    avg_test_loss = avg_test_loss.item()
    
    # Measure how long the validation run took.
    test_time = format_time(time.time() - t0)
    
    print("  Test Loss: {0:.3f}".format(avg_test_loss))
    print("  Test took: {:}".format(test_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss generator': avg_train_loss_g,
            'Training Loss discriminator': avg_train_loss_d,
            'Valid. Loss': avg_test_loss,
            'Valid. Accur.': test_accuracy,
            'Training Time': training_time,
            'Test Time': test_time
        }
    )


======== Epoch 1 / 1 ========
Training...
  Batch    10  of  12,567.    Elapsed: 0:00:03.
  Batch    20  of  12,567.    Elapsed: 0:00:06.
  Batch    30  of  12,567.    Elapsed: 0:00:09.
  Batch    40  of  12,567.    Elapsed: 0:00:12.
  Batch    50  of  12,567.    Elapsed: 0:00:15.
  Batch    60  of  12,567.    Elapsed: 0:00:18.
  Batch    70  of  12,567.    Elapsed: 0:00:21.
  Batch    80  of  12,567.    Elapsed: 0:00:24.
  Batch    90  of  12,567.    Elapsed: 0:00:27.
  Batch   100  of  12,567.    Elapsed: 0:00:30.
  Batch   110  of  12,567.    Elapsed: 0:00:33.
  Batch   120  of  12,567.    Elapsed: 0:00:36.
  Batch   130  of  12,567.    Elapsed: 0:00:39.
  Batch   140  of  12,567.    Elapsed: 0:00:42.
  Batch   150  of  12,567.    Elapsed: 0:00:45.
  Batch   160  of  12,567.    Elapsed: 0:00:48.
  Batch   170  of  12,567.    Elapsed: 0:00:51.
  Batch   180  of  12,567.    Elapsed: 0:00:54.
  Batch   190  of  12,567.    Elapsed: 0:00:57.
  Batch   200  of  12,567.    Elapsed: 0:01:0

  Batch 1,710  of  12,567.    Elapsed: 0:08:29.
  Batch 1,720  of  12,567.    Elapsed: 0:08:32.
  Batch 1,730  of  12,567.    Elapsed: 0:08:35.
  Batch 1,740  of  12,567.    Elapsed: 0:08:38.
  Batch 1,750  of  12,567.    Elapsed: 0:08:41.
  Batch 1,760  of  12,567.    Elapsed: 0:08:44.
  Batch 1,770  of  12,567.    Elapsed: 0:08:47.
  Batch 1,780  of  12,567.    Elapsed: 0:08:50.
  Batch 1,790  of  12,567.    Elapsed: 0:08:53.
  Batch 1,800  of  12,567.    Elapsed: 0:08:56.
  Batch 1,810  of  12,567.    Elapsed: 0:08:59.
  Batch 1,820  of  12,567.    Elapsed: 0:09:02.
  Batch 1,830  of  12,567.    Elapsed: 0:09:05.
  Batch 1,840  of  12,567.    Elapsed: 0:09:08.
  Batch 1,850  of  12,567.    Elapsed: 0:09:11.
  Batch 1,860  of  12,567.    Elapsed: 0:09:14.
  Batch 1,870  of  12,567.    Elapsed: 0:09:17.
  Batch 1,880  of  12,567.    Elapsed: 0:09:20.
  Batch 1,890  of  12,567.    Elapsed: 0:09:23.
  Batch 1,900  of  12,567.    Elapsed: 0:09:26.
  Batch 1,910  of  12,567.    Elapsed: 0

  Batch 3,420  of  12,567.    Elapsed: 0:17:01.
  Batch 3,430  of  12,567.    Elapsed: 0:17:04.
  Batch 3,440  of  12,567.    Elapsed: 0:17:07.
  Batch 3,450  of  12,567.    Elapsed: 0:17:10.
  Batch 3,460  of  12,567.    Elapsed: 0:17:13.
  Batch 3,470  of  12,567.    Elapsed: 0:17:16.
  Batch 3,480  of  12,567.    Elapsed: 0:17:19.
  Batch 3,490  of  12,567.    Elapsed: 0:17:22.
  Batch 3,500  of  12,567.    Elapsed: 0:17:25.
  Batch 3,510  of  12,567.    Elapsed: 0:17:28.
  Batch 3,520  of  12,567.    Elapsed: 0:17:31.
  Batch 3,530  of  12,567.    Elapsed: 0:17:33.
  Batch 3,540  of  12,567.    Elapsed: 0:17:36.
  Batch 3,550  of  12,567.    Elapsed: 0:17:39.
  Batch 3,560  of  12,567.    Elapsed: 0:17:42.
  Batch 3,570  of  12,567.    Elapsed: 0:17:45.
  Batch 3,580  of  12,567.    Elapsed: 0:17:48.
  Batch 3,590  of  12,567.    Elapsed: 0:17:51.
  Batch 3,600  of  12,567.    Elapsed: 0:17:54.
  Batch 3,610  of  12,567.    Elapsed: 0:17:58.
  Batch 3,620  of  12,567.    Elapsed: 0

  Batch 5,130  of  12,567.    Elapsed: 0:25:31.
  Batch 5,140  of  12,567.    Elapsed: 0:25:34.
  Batch 5,150  of  12,567.    Elapsed: 0:25:37.
  Batch 5,160  of  12,567.    Elapsed: 0:25:40.
  Batch 5,170  of  12,567.    Elapsed: 0:25:43.
  Batch 5,180  of  12,567.    Elapsed: 0:25:46.
  Batch 5,190  of  12,567.    Elapsed: 0:25:49.
  Batch 5,200  of  12,567.    Elapsed: 0:25:52.
  Batch 5,210  of  12,567.    Elapsed: 0:25:54.
  Batch 5,220  of  12,567.    Elapsed: 0:25:57.
  Batch 5,230  of  12,567.    Elapsed: 0:26:00.
  Batch 5,240  of  12,567.    Elapsed: 0:26:03.
  Batch 5,250  of  12,567.    Elapsed: 0:26:06.
  Batch 5,260  of  12,567.    Elapsed: 0:26:09.
  Batch 5,270  of  12,567.    Elapsed: 0:26:12.
  Batch 5,280  of  12,567.    Elapsed: 0:26:15.
  Batch 5,290  of  12,567.    Elapsed: 0:26:18.
  Batch 5,300  of  12,567.    Elapsed: 0:26:21.
  Batch 5,310  of  12,567.    Elapsed: 0:26:24.
  Batch 5,320  of  12,567.    Elapsed: 0:26:27.
  Batch 5,330  of  12,567.    Elapsed: 0

  Batch 6,840  of  12,567.    Elapsed: 0:34:00.
  Batch 6,850  of  12,567.    Elapsed: 0:34:03.
  Batch 6,860  of  12,567.    Elapsed: 0:34:06.
  Batch 6,870  of  12,567.    Elapsed: 0:34:09.
  Batch 6,880  of  12,567.    Elapsed: 0:34:12.
  Batch 6,890  of  12,567.    Elapsed: 0:34:15.
  Batch 6,900  of  12,567.    Elapsed: 0:34:18.
  Batch 6,910  of  12,567.    Elapsed: 0:34:21.
  Batch 6,920  of  12,567.    Elapsed: 0:34:24.
  Batch 6,930  of  12,567.    Elapsed: 0:34:27.
  Batch 6,940  of  12,567.    Elapsed: 0:34:30.
  Batch 6,950  of  12,567.    Elapsed: 0:34:33.
  Batch 6,960  of  12,567.    Elapsed: 0:34:36.
  Batch 6,970  of  12,567.    Elapsed: 0:34:39.
  Batch 6,980  of  12,567.    Elapsed: 0:34:42.
  Batch 6,990  of  12,567.    Elapsed: 0:34:45.
  Batch 7,000  of  12,567.    Elapsed: 0:34:48.
  Batch 7,010  of  12,567.    Elapsed: 0:34:51.
  Batch 7,020  of  12,567.    Elapsed: 0:34:54.
  Batch 7,030  of  12,567.    Elapsed: 0:34:57.
  Batch 7,040  of  12,567.    Elapsed: 0

  Batch 8,550  of  12,567.    Elapsed: 0:42:33.
  Batch 8,560  of  12,567.    Elapsed: 0:42:36.
  Batch 8,570  of  12,567.    Elapsed: 0:42:39.
  Batch 8,580  of  12,567.    Elapsed: 0:42:42.
  Batch 8,590  of  12,567.    Elapsed: 0:42:46.
  Batch 8,600  of  12,567.    Elapsed: 0:42:49.
  Batch 8,610  of  12,567.    Elapsed: 0:42:52.
  Batch 8,620  of  12,567.    Elapsed: 0:42:55.
  Batch 8,630  of  12,567.    Elapsed: 0:42:58.
  Batch 8,640  of  12,567.    Elapsed: 0:43:01.
  Batch 8,650  of  12,567.    Elapsed: 0:43:04.
  Batch 8,660  of  12,567.    Elapsed: 0:43:07.
  Batch 8,670  of  12,567.    Elapsed: 0:43:10.
  Batch 8,680  of  12,567.    Elapsed: 0:43:13.
  Batch 8,690  of  12,567.    Elapsed: 0:43:16.
  Batch 8,700  of  12,567.    Elapsed: 0:43:19.
  Batch 8,710  of  12,567.    Elapsed: 0:43:22.
  Batch 8,720  of  12,567.    Elapsed: 0:43:25.
  Batch 8,730  of  12,567.    Elapsed: 0:43:28.
  Batch 8,740  of  12,567.    Elapsed: 0:43:31.
  Batch 8,750  of  12,567.    Elapsed: 0

  Batch 10,260  of  12,567.    Elapsed: 0:51:18.
  Batch 10,270  of  12,567.    Elapsed: 0:51:21.
  Batch 10,280  of  12,567.    Elapsed: 0:51:24.
  Batch 10,290  of  12,567.    Elapsed: 0:51:27.
  Batch 10,300  of  12,567.    Elapsed: 0:51:30.
  Batch 10,310  of  12,567.    Elapsed: 0:51:33.
  Batch 10,320  of  12,567.    Elapsed: 0:51:37.
  Batch 10,330  of  12,567.    Elapsed: 0:51:40.
  Batch 10,340  of  12,567.    Elapsed: 0:51:43.
  Batch 10,350  of  12,567.    Elapsed: 0:51:46.
  Batch 10,360  of  12,567.    Elapsed: 0:51:49.
  Batch 10,370  of  12,567.    Elapsed: 0:51:52.
  Batch 10,380  of  12,567.    Elapsed: 0:51:55.
  Batch 10,390  of  12,567.    Elapsed: 0:51:58.
  Batch 10,400  of  12,567.    Elapsed: 0:52:01.
  Batch 10,410  of  12,567.    Elapsed: 0:52:04.
  Batch 10,420  of  12,567.    Elapsed: 0:52:07.
  Batch 10,430  of  12,567.    Elapsed: 0:52:10.
  Batch 10,440  of  12,567.    Elapsed: 0:52:13.
  Batch 10,450  of  12,567.    Elapsed: 0:52:16.
  Batch 10,460  of  

  Batch 11,940  of  12,567.    Elapsed: 0:59:43.
  Batch 11,950  of  12,567.    Elapsed: 0:59:46.
  Batch 11,960  of  12,567.    Elapsed: 0:59:49.
  Batch 11,970  of  12,567.    Elapsed: 0:59:52.
  Batch 11,980  of  12,567.    Elapsed: 0:59:55.
  Batch 11,990  of  12,567.    Elapsed: 0:59:58.
  Batch 12,000  of  12,567.    Elapsed: 1:00:01.
  Batch 12,010  of  12,567.    Elapsed: 1:00:04.
  Batch 12,020  of  12,567.    Elapsed: 1:00:07.
  Batch 12,030  of  12,567.    Elapsed: 1:00:10.
  Batch 12,040  of  12,567.    Elapsed: 1:00:13.
  Batch 12,050  of  12,567.    Elapsed: 1:00:16.
  Batch 12,060  of  12,567.    Elapsed: 1:00:19.
  Batch 12,070  of  12,567.    Elapsed: 1:00:22.
  Batch 12,080  of  12,567.    Elapsed: 1:00:24.
  Batch 12,090  of  12,567.    Elapsed: 1:00:27.
  Batch 12,100  of  12,567.    Elapsed: 1:00:30.
  Batch 12,110  of  12,567.    Elapsed: 1:00:33.
  Batch 12,120  of  12,567.    Elapsed: 1:00:36.
  Batch 12,130  of  12,567.    Elapsed: 1:00:39.
  Batch 12,140  of  

c:\users\komol\appdata\local\programs\python\python37\lib\site-packages\torch\serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Discriminator. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "



Running Test...
  Accuracy: 0.540
  Test Loss: 0.690
  Test took: 0:00:11


In [60]:
print("")
print("Running Test...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
transformer.eval() #maybe redundant
discriminator.eval()

# Tracking variables 
total_test_accuracy = 0

total_test_loss = 0
nb_test_steps = 0

all_preds = []
all_labels_ids = []

#loss
nll_loss = torch.nn.CrossEntropyLoss(ignore_index=-1)

# Evaluate data for one epoch
for batch in test_dataloader:
    
    # Unpack this training batch from our dataloader. 
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        
        model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
        hidden_states = model_outputs[-1]
        _, logits, probs = discriminator(hidden_states)
        ###log_probs = F.log_softmax(probs[:,1:], dim=-1)
        filtered_logits = logits[:,0:-1]
        # Accumulate the test loss.
        total_test_loss += nll_loss(filtered_logits, b_labels)
        
    # Accumulate the predictions and the input labels
    _, preds = torch.max(filtered_logits, 1)
    all_preds += preds.detach().cpu()
    all_labels_ids += b_labels.detach().cpu()

# Report the final accuracy for this validation run.
all_preds = torch.stack(all_preds).numpy()
all_labels_ids = torch.stack(all_labels_ids).numpy()
test_accuracy = np.sum(all_preds == all_labels_ids) / len(all_preds)
print("  Accuracy: {0:.3f}".format(test_accuracy))

# Calculate the average loss over all of the batches.
avg_test_loss = total_test_loss / len(test_dataloader)
avg_test_loss = avg_test_loss.item()

# Measure how long the validation run took.
test_time = format_time(time.time() - t0)
  
print("  Test Loss: {0:.3f}".format(avg_test_loss))
print("  Test took: {:}".format(test_time))


Running Test...
  Accuracy: 0.540
  Test Loss: 0.690
  Test took: 0:00:11
